# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
#load environment and setting working ditectory to the .env file path
import os
os.chdir('../../02_activities/assignments/')
%load_ext dotenv
%dotenv 



In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\pocho\miniconda3\envs\scale2prod\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
ft_dir = os.getenv("PRICE_DATA")
print("Directory path:", ft_dir)

ft_glob = glob(ft_dir+"/**/*.parquet/*.parquet")
df = dd.read_parquet(ft_glob).reset_index()
print(type(df))
df.head()



Directory path: ../../05_src/data/prices/
<class 'dask_expr._collection.DataFrame'>


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
0,A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613014,4674353,Health Care,Life Sciences Tools & Services,2000
1,A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281464,4765083,Health Care,Life Sciences Tools & Services,2000
2,A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782791,5758642,Health Care,Life Sciences Tools & Services,2000
3,A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344158,2534434,Health Care,Life Sciences Tools & Services,2000
4,A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372845,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
# Adding lags for 'Close'
df['Close_lag'] = df.groupby('ticker')['Close'].shift(1)

# Adding returns based on 'Close'
df['returns'] = (df['Close'] / df['Close_lag']) - 1

# Adding hi_lo_range
df['hi_lo_range'] = df['High'] - df['Low']

# Assigning the result to dd_feat
dd_feat = df

#Visualizing dd_feat to make sure calculations are corect
dd_feat

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
import pandas as pd

# Converting the Dask DataFrame to a Pandas DataFrame using the compute() method
pandas_feat = dd_feat.compute()

# Adding a rolling average return calculation with a window of 10 days Grouping by ticker to calculate rolling mean for each ticker separately
pandas_feat['rolling_avg_return'] = pandas_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(window=10).mean())

# Display the data frame
pandas_feat



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Answer**: 
It was not necessary to convert to pandas. For Scalability purposes if dealing with large datasets it would have been better to do it in dask to take advantage of its parallel computation capabilities.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.